In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import random

headers = { 
	"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36", 
	"Referer": "https://www.domain.com.au/"
}

base_url = "https://www.domain.com.au/rent/"
errCount = 0
listing_ids = []

for i in range(1,6):
    if i > 1:
        url = f"{base_url}?page={i}"
    else:
        url = base_url
    
    response = requests.get(url, headers=headers)
    if response.ok:
        soup = BeautifulSoup(response.content, 'html')
        elements_with_class = soup.find_all(class_='css-1qp9106')
        
        for element in elements_with_class:
            data_testid_value = element.get('data-testid')
            if data_testid_value:
                listing_id = re.sub(r'\D', '', data_testid_value)
                listing_ids.append(listing_id)

    else:
        print(response)
        errCount += 1


In [2]:
headers = {
    'X-API-Key': 'key_07e6c61e2af44856103c7a58aa5339aa'
    }

base_url = 'https://api.domain.com.au/v1/me'

authenticate = requests.get(base_url, headers=headers)

authenticate.status_code


200

In [100]:
import pandas as pd

base_url = 'https://api.domain.com.au/v1/listings/'
url = f"{base_url}{listing_ids[0]}"
response = requests.get(url, headers=headers)

if response.ok:
    listing = response.json()
else:
    raise RuntimeError

titles = list(listing.keys())

df = pd.DataFrame(columns = titles)

df = df.drop(columns=['status', 'geoLocation', 'isNewDevelopment', 'media', 'channel', 
                      'rentalDetails', 'advertiserIdentifiers', 'inspectionDetails',
                      'saleMode', 'description'])

for i in listing_ids:
    url = f'{base_url}{i}'
    response = requests.get(url, headers=headers)
    if response.ok:
        listing = response.json()
        df.loc[len(df.index)] = listing


In [101]:
df['apmIdentifiers'] = df['addressParts'].apply(lambda x: x['suburb'])
df = df.rename(columns = {'apmIdentifiers': 'suburb'})

df['addressParts'] = df['addressParts'].apply(lambda x: x['displayAddress'])
df = df.rename(columns={'addressParts': 'fullAddress'})

df['priceDetails'] = df['priceDetails'].apply(lambda x: x['displayPrice'])
df = df.rename(columns={'priceDetails': 'displayPrice'})

In [20]:
url = f"{base_url}{listing_ids[50]}"

print(url)
response = requests.get(url, headers=headers)

if response.ok:
    listing = response.json()

listing['priceDetails']['displayPrice']

https://api.domain.com.au/v1/listings/17163068


'$1600 Per Week'

,objective,id,fullAddress,suburb,bathrooms,bedrooms,carspaces,dateAvailable,dateUpdated,dateListed,headline,displayPrice,propertyId,propertyTypes,seoUrl
0,rent,17163316,"51 Molden Street, Bentleigh East VIC 3165",Bentleigh East,4,4,2,2024-08-15T00:00:00+10:00,2024-08-15T05:42:59.987Z,2024-08-15T05:13:26Z,The Ultimate in Family Luxury in Zone! *Open S...,"$1,450.00",QR-5317-LK,[townhouse],https://www.domain.com.au/51-molden-street-ben...
1,rent,16609817,"G01A/72 Ascot Vale Road, Flemington VIC 3031",Flemington,2,2,1,2024-09-13T00:00:00+10:00,2024-08-15T06:39:53.78Z,2024-08-15T05:10:57Z,Brand New Apartment Featuring Top Class Amenities,$630.00 per week,NaN,[apartmentUnitFlat],https://www.domain.com.au/g01a-72-ascot-vale-r...
2,rent,17163300,"701/8 Garfield Street, Richmond VIC 3121",Richmond,2,2,2,NaN,2024-08-15T05:18:49.917Z,2024-08-15T05:08:50Z,MAGNIFICENT VIEWS TO MATCH A MAGNIFICENT LIFES...,$850 PER WEEK,XZ-2424-PC,[apartmentUnitFlat],https://www.domain.com.au/701-8-garfield-stree...
3,rent,17163298,"B803/87 Bay Street, Glebe NSW 2037",Glebe,2,2,1,2024-08-15T00:00:00+10:00,2024-08-15T05:40:03.813Z,2024-08-15T05:08:23Z,"Stunning Sun-Drenched, Two Bedroom Apartment i...","$1,200 pw",NaN,[apartmentUnitFlat],https://www.domain.com.au/b803-87-bay-street-g...
4,rent,17163296,"602/12 Yarra Street, South Yarra VIC 3141",South Yarra,2,2,1,2024-09-12T00:00:00+10:00,2024-08-15T05:24:27.953Z,2024-08-15T05:08:17Z,PLEASE CLICK THE BELOW 'REQUEST AN INSPECTION'...,$780.00,VO-5616-MT,[apartmentUnitFlat],https://www.domain.com.au/602-12-yarra-street-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,rent,17162756,"19 Oak Street, Lugarno NSW 2210",Lugarno,2,4,2,2024-08-14T00:00:00+10:00,2024-08-15T06:54:21.5Z,2024-08-15T02:43:52Z,Spacious and Serene 4-Bedroom Family Home with...,"$1,150 pw",XA-8457-WE,[house],https://www.domain.com.au/19-oak-street-lugarn...
96,rent,17162753,"G07/10-14 Arthur Street, Marrickville NSW 2204",Marrickville,1,1,1,NaN,2024-08-15T02:42:29.63Z,2024-08-15T02:42:29Z,Near New Groundfloor One bedroom with Courtyard!,$810 per week,NaN,[apartmentUnitFlat],https://www.domain.com.au/g07-10-14-arthur-str...
97,rent,17162745,"39 Palm Beach Crescent, Mount Waverley VIC 3149",Mount Waverley,2,3,2,2024-09-16T00:00:00+10:00,2024-08-15T02:40:32.847Z,2024-08-15T02:40:32Z,MODERN THREE BEDROOM HOME IN MWSC CATCHMENT,$700.00,YL-4721-CT,[house],https://www.domain.com.au/39-palm-beach-cresce...
98,rent,17162742,"404/45 Shelley Street, Sydney NSW 2000",Sydney,2,2,0,2024-08-12T00:00:00+10:00,2024-08-15T02:40:19.523Z,2024-08-15T02:40:19Z,Fully Furnished Beautiful Two Bedroom Apartment,"$1,200",CP-2953-OJ,[apartmentUnitFlat],https://www.domain.com.au/404-45-shelley-stree...


In [42]:
from minio import Minio
# Initialize MinIO client
client = Minio('localhost:9000',
               access_key='accessKey',
               secret_key='secretKey',
               secure=False)
# Make a bucket

df.to_csv(r'./output.csv')

client.fput_object('domain-data', 'output.csv', 'output.csv')


In [21]:
from types_realestate import SearchCoordinate



Hello World!


In [1]:
client = Minio('localhost:9000',
               access_key='accessKey',
               secret_key='secretKey',
               secure=False)

client.list_buckets()

NameError: name 'Minio' is not defined